# Projeto Final - Semantix Academy
### Turma 202

<p>Arquivo salvo na minha maquina.</p>
<p>Descompactei os arquivos</p>
<p>Arquivos enviados para o Spark/input/covid19</p>
<p>Entrei no container namenode bash</p>
<p>E realizei o -put dos arquivos para dentro do hdfs</p>
<p>
root@namenode:/input# cd covid19<br>
root@namenode:/input/covid19# ls<br>
HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv  HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv  HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv<br>
root@namenode:/input/covid19# hdfs dfs -mkdir /covid19<br>
root@namenode:/input/covid19# hdfs dfs -put * /covid19<br>
root@namenode:/input/covid19# hdfs dfs -ls /covid19<br>
Found 4 items<br>
-rw-r--r--   3 root supergroup   62492959 2022-04-22 17:06 /covid19/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv<br>
-rw-r--r--   3 root supergroup   76520681 2022-04-22 17:06 /covid19/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv<br>
-rw-r--r--   3 root supergroup   91120916 2022-04-22 17:06 /covid19/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv<br>
-rw-r--r--   3 root supergroup    3046774 2022-04-22 17:06 /covid19/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv<br>
</p>    

In [1]:
spark.sparkContext.setLogLevel("INFO")


In [2]:
spark.catalog.listDatabases()

[Database(name='covid19', description='Projeto-Semantix', locationUri='hdfs://namenode:8020/user/hive/warehouse/covid19.db'),
 Database(name='default', description='Default Hive database', locationUri='hdfs://namenode:8020/user/hive/warehouse')]

In [3]:
spark.catalog.setCurrentDatabase("covid19")

In [4]:
spark.catalog.currentDatabase()

'covid19'

In [5]:
spark.catalog.listTables()

[]

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta

estrutura_colunas = [
    
    StructField("regiao", StringType()),
    StructField("estado", StringType()),
    StructField("municipio", StringType()),
    StructField("coduf", StringType()),
    StructField("codmun", IntegerType()),
    StructField("codRegiaoSaude", IntegerType()),
    StructField("nomeRegiaoSaude", StringType()),
    StructField("data", StringType()),
    StructField("semanaEpi", IntegerType()),
    StructField("populacaoTCU2019", IntegerType()),
    StructField("casosAcumulado", IntegerType()),
    StructField("casosNovos", IntegerType()),
    StructField("obitosAcumulado", StringType()),
    StructField("obitosNovos", IntegerType()),
    StructField("Recuperadosnovos", IntegerType()),
    StructField("emAcompanhamentoNovos", IntegerType()),
    StructField("interior/metropolitana", StringType())
]

schema_names = StructType(estrutura_colunas)

df = spark.read.csv("/covid19/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv", header='true',sep=';', schema = schema_names)
df = df.withColumn("incidenciaCasos", format_number(col("casosAcumulado")/(col("populacaoTCU2019")/100000),2))
df = df.withColumn("incidenciaObitos", format_number(col("obitosAcumulado")/(col("populacaoTCU2019")/100000),2))
df = df.withColumn("letalidade", format_number((col("obitosAcumulado")/col("casosAcumulado"))*100,2))

df.show(5)
df.printSchema()
df.count()

+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+---------------+----------------+----------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|incidenciaCasos|incidenciaObitos|letalidade|
+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+---------------+----------------+----------+
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-25|        9|       210147125|             0|         0|              0|          0|            null|                 n

714481

Na hora que efetuei a leitura dos 4 arquivos para criação da tabela,<br>
meu pc nao aguentou o tamanho dos arquivos e estava travando.<br>
Com isso fiz a leitura de um arquivo apenas: "HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv"<br>


In [8]:
#Particionando os dados por "Municipio"
df.write.mode("overwrite").partitionBy("municipio").saveAsTable("DadosCovid19")

In [9]:
#Demonstrando a Criação da Tabela
spark.catalog.listTables()

[Table(name='dadoscovid19', database='covid19', description=None, tableType='MANAGED', isTemporary=False)]

In [41]:
maxData = df.select("regiao","data").filter(col("regiao") == "Brasil").agg(max("data")).collect()[0]["max(data)"]

In [17]:
df.show(5)

+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+---------------+----------------+----------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|incidenciaCasos|incidenciaObitos|letalidade|
+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+---------------+----------------+----------+
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-25|        9|       210147125|             0|         0|              0|          0|            null|                 n

In [42]:
Painel1 = df.filter(col("regiao") == "Brasil").\
            filter(col("data") == maxData).\
            select("Recuperadosnovos","emAcompanhamentoNovos") 
Painel2 = df.filter(col("regiao") == "Brasil").\
            filter(col("data") == maxData).\
            select("casosAcumulado","casosNovos","incidenciaCasos")
Painel3 = df.filter(col("regiao") == "Brasil").\
            filter(col("data") == maxData).\
            select("obitosAcumulado","obitosNovos","incidenciaObitos","letalidade")


In [45]:
Painel1.show()

+----------------+---------------------+
|Recuperadosnovos|emAcompanhamentoNovos|
+----------------+---------------------+
|         1844051|               725959|
+----------------+---------------------+



In [46]:
Painel2.show()

+--------------+----------+---------------+
|casosAcumulado|casosNovos|incidenciaCasos|
+--------------+----------+---------------+
|       2662485|     52383|       1,266.96|
+--------------+----------+---------------+



In [48]:
Painel3.show()

+---------------+-----------+----------------+----------+
|obitosAcumulado|obitosNovos|incidenciaObitos|letalidade|
+---------------+-----------+----------------+----------+
|          92475|       1212|           44.00|      3.47|
+---------------+-----------+----------------+----------+



In [49]:
Painel1.write.mode("overwrite").saveAsTable("Painel1Covid19")

In [50]:
spark.catalog.listTables()

[Table(name='dadoscovid19', database='covid19', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='painel1covid19', database='covid19', description=None, tableType='MANAGED', isTemporary=False)]

In [51]:
Painel2.write.parquet("/covid19/Painel2", compression='snappy')

In [52]:
!hdfs dfs -ls /covid19/Painel2

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-04-22 19:46 /covid19/Painel2/_SUCCESS
-rw-r--r--   2 root supergroup        927 2022-04-22 19:46 /covid19/Painel2/part-00000-9fdcad14-6a91-4604-ae8a-f9e68a661100-c000.snappy.parquet


In [53]:
kafka_obitos_acumulado = Painel3.select("obitosAcumulado")\
                       .withColumnRenamed("obitosAcumulado","value")\
                       .withColumn("value",col("value").cast(StringType()))

kafka_obitos_acumulado.write.format('kafka')\
              .option("kafka.bootstrap.servers","kafka:9092")\
              .option("topic","topic-kafka-obitos")\
              .save()

kafka_novosObitos = Painel3.select("obitosNovos")\
                       .withColumnRenamed("obitosNovos","value")\
                       .withColumn("value",col("value").cast(StringType()))

kafka_novosObitos.write.format('kafka')\
              .option("kafka.bootstrap.servers","kafka:9092")\
              .option("topic","topic-kafka-NovosObitos")\
              .save()

kafka_incidenciaObitos = Painel3.select("incidenciaObitos")\
                       .withColumnRenamed("incidenciaObitos","value")\
                       .withColumn("value",col("value").cast(StringType()))

kafka_incidenciaObitos.write.format('kafka')\
              .option("kafka.bootstrap.servers","kafka:9092")\
              .option("topic","topic-kafka-incidenciaObitos")\
              .save()

kafka_letalidade = Painel3.select("letalidade")\
                       .withColumnRenamed("letalidade","value")\
                       .withColumn("value",col("value").cast(StringType()))

kafka_incidenciaObitos.write.format('kafka')\
              .option("kafka.bootstrap.servers","kafka:9092")\
              .option("topic","topic-kafka-letalidade")\
              .save()

In [75]:
df.select("regiao","estado","casosAcumulado","obitosAcumulado","incidenciaCasos","incidenciaObitos").filter(col("regiao")!= "Brasil").groupBy("regiao").agg(sum("casosAcumulado").alias("Casos"),\
                                                                                                    sum("obitosAcumulado").alias("Óbtos"),\
                                                                                                    format_number(avg("incidenciaCasos"),1).alias("Incidência"),\
                                                                                                    format_number(avg("incidenciaObitos"),1).alias("Mortalidade"),\
                                                                                                    ).show(10)


+------------+--------+---------+----------+-----------+
|      regiao|   Casos|    Óbtos|Incidência|Mortalidade|
+------------+--------+---------+----------+-----------+
|    Nordeste|67203769|2704773.0|     146.1|        7.9|
|         Sul|12644537| 275955.0|     102.1|        2.7|
|     Sudeste|70142488|3985102.0|     103.4|        4.8|
|Centro-Oeste|14068947| 284181.0|      96.4|        3.6|
|       Norte|35543862|1347798.0|     165.1|       13.8|
+------------+--------+---------+----------+-----------+



In [76]:
from pandas.io.json import json_normalize
import json
import requests
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SQLContext

In [77]:
Texto = Painel3.collect()[0].asDict()
Texto

{'obitosAcumulado': '92475',
 'obitosNovos': 1212,
 'incidenciaObitos': '44.00',
 'letalidade': '3.47'}

In [ ]:
URL = "https://desafiosemantix.es.us-west1.gcp.cloud.es.io:9243/dadospainel2/_doc/1"
headers = {'Content-type': 'application/json; charset=utf8'}
response = requests.post(URL, auth=('elastic', 'gPbGxozAcdl5rHBeXFdlnJdY'),json=doc,headers=headers)
response.encoding
response.text
response.json()